## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import time


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,6.96,55,0,4.85,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,69.80,87,3,11.21,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,55.11,62,100,21.12,overcast clouds
3,3,Palu,ID,-0.8917,119.8707,76.37,94,100,2.55,overcast clouds
4,4,Port Moresby,PG,-9.4431,147.1797,80.02,83,20,5.75,few clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                       (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Palu,ID,-0.8917,119.8707,76.37,94,100,2.55,overcast clouds
4,4,Port Moresby,PG,-9.4431,147.1797,80.02,83,20,5.75,few clouds
5,5,Bengkulu,ID,-3.8004,102.2655,74.88,87,90,4.59,overcast clouds
7,7,Phan Thiet,VN,10.9333,108.1000,77.14,83,90,4.56,overcast clouds
9,9,Carnarvon,AU,-24.8667,113.6333,76.41,69,99,19.28,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
688,688,Alyangula,AU,-13.8483,136.4192,85.53,63,76,8.93,broken clouds
692,692,Butterworth,MY,5.3991,100.3638,80.64,78,20,2.30,few clouds
694,694,Parrita,CR,9.5201,-84.3225,83.57,68,57,4.34,broken clouds
695,695,Inirida,CO,3.8653,-67.9239,75.25,94,72,1.86,broken clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
for index, row in preferred_cities_df.iterrows():
    country_len = len(str(row['Country']))
    
    if country_len == 3:
        clean_df = preferred_cities_df.drop(index)
        print(f'Dropped {index}')

Dropped 56


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Palu,ID,76.37,overcast clouds,-0.8917,119.8707,
4,Port Moresby,PG,80.02,few clouds,-9.4431,147.1797,
5,Bengkulu,ID,74.88,overcast clouds,-3.8004,102.2655,
7,Phan Thiet,VN,77.14,overcast clouds,10.9333,108.1000,
9,Carnarvon,AU,76.41,overcast clouds,-24.8667,113.6333,
12,San Policarpo,PH,79.77,overcast clouds,12.1791,125.5072,
14,Bandarbeyla,SO,75.58,few clouds,9.4942,50.8122,
15,Omboue,GA,78.71,overcast clouds,-1.5746,9.2618,
16,Yulara,AU,80.11,clear sky,-25.2406,130.9889,
20,Avarua,CK,86.05,scattered clouds,-21.2078,-159.7750,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
count=0
# 6b. Iterate through the hotel DataFrame.
print('Collecting Hotel Information...')

for index,row in clean_df.iterrows():
    
    count += 1
    
    if (count%50 == 0) & (count >=50):
        print('Maximum API request limit,waiting 60 seconds...')
        time.sleep(60)

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except:
        print('Hotel Not Found, skipping...')
        
print('All Done')

Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
All Done


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [26]:
for index, row in hotel_df.iterrows():
    hotel_name = len(row['Hotel Name'])
    
    if hotel_name == 0:
        hotel_df = hotel_df.drop(index)

In [27]:
clean_hotel_df = hotel_df.copy()

In [28]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>and<dd>{Max Temp} °F</dd> 
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
hotel_marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(hotel_marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))